In [4]:
# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
from cellpose import models, core, io, utils

import pathlib

In [5]:
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)
logger_setup();

2022-06-20 15:31:31,710 [INFO] ** TORCH CUDA version installed and working. **
>>> GPU activated? 1
2022-06-20 15:31:31,712 [INFO] WRITING LOG OUTPUT TO /home/roshankern/.cellpose/run.log


In [6]:
def segment_training_data(preproc_training_path, save_path, cellpose_model):
    """
    Saves 
    """
    for plate_path in preproc_training_path.iterdir():
        for well_path in plate_path.iterdir():
            for frame_path in well_path.iterdir():
                for file_path in frame_path.iterdir():
                    segmented_save_dir = pathlib.Path(f"{save_path}/{plate_path.name}/{well_path.name}/{frame_path.name}")
                    print(f"Segmenting: {file_path}")
                    if segmented_save_dir.exists():
                        print("Movie has already been segmented!")
                    else:
                        channels = [0,0]
                        frame_image = io.imread(file_path)
                        masks, flows, styles, diams = cellpose_model.eval(frame_image, diameter=0, channels=channels, flow_threshold=0.8)
                        outlines = utils.outlines_list(masks)
                        for outline in outlines:
                            centroid = outline.mean(axis=0)
                            print(centroid)
                        segmented_save_dir.mkdir(parents=True, exist_ok=False)
                        return

load_path = pathlib.Path("../1.preprocess_data/labeled_frames_preprocessed/")
save_path = pathlib.Path("segmented/")
cellpose_model = models.Cellpose(gpu=True, model_type='cyto')
segment_training_data(load_path, save_path, cellpose_model)

2022-06-20 15:31:31,739 [INFO] ** TORCH CUDA version installed and working. **
2022-06-20 15:31:31,739 [INFO] >>>> using GPU
2022-06-20 15:31:31,740 [INFO] >> cyto << model set to be used
2022-06-20 15:31:31,837 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
Segmenting: ../1.preprocess_data/labeled_frames_preprocessed/LT0043_48/166/48/LT0043_48_166_48.tif
Movie has already been segmented!
Segmenting: ../1.preprocess_data/labeled_frames_preprocessed/LT0043_48/166/55/LT0043_48_166_55.tif
Movie has already been segmented!
Segmenting: ../1.preprocess_data/labeled_frames_preprocessed/LT0043_48/166/36/LT0043_48_166_36.tif
Movie has already been segmented!
Segmenting: ../1.preprocess_data/labeled_frames_preprocessed/LT0043_48/166/74/LT0043_48_166_74.tif
Movie has already been segmented!
Segmenting: ../1.preprocess_data/labeled_frames_preprocessed/LT0043_48/166/44/LT0043_48_166_44.tif
2022-06-20 15:31:31,840 [INFO] ~~~ ESTIMATING CELL DIAMETER(S) ~~~
2022-06